In [1]:
import bs4 as bs
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
from collections import Counter
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

a, b = process_data_for_labels('XOM')

def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [2]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)

    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)] ))
    #print(df.head())
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:',Counter(str_vals))
    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)
    X = df_vals.values
    y = df['{}_target'.format(ticker)].values

    return X,y,df
#extract_featuresets('XOM')

In [3]:
def do_ml(ticker):
    X, y, df = extract_featuresets(ticker)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    clf = VotingClassifier([('lsvc', svm.LinearSVC()),
                            ('knn', neighbors.KNeighborsClassifier()),
                            ('rfor', RandomForestClassifier())])

    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:', confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:', Counter(predictions))
    print()
    print()
    return confidence


# examples of running:
do_ml('XOM')
do_ml('AAPL')
do_ml('ABT')

Data spread: Counter({'-1': 1347, '1': 1224, '0': 59})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.5045592705167173
predicted class counts: Counter({-1: 434, 1: 223, 0: 1})


Data spread: Counter({'-1': 1376, '1': 1201, '0': 53})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.5501519756838906
predicted class counts: Counter({-1: 376, 1: 281, 0: 1})


Data spread: Counter({'-1': 1340, '1': 1215, '0': 75})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.47416413373860183
predicted class counts: Counter({-1: 401, 1: 256, 0: 1})




0.47416413373860183

In [ ]:
from statistics import mean

with open("sp500tickers.pickle","rb") as f:
    tickers = pickle.load(f)

accuracies = []
for count,ticker in enumerate(tickers):
    for ticker in tickers:
        ticker = ticker.strip("\n")
        if "." in ticker:
            ticker = ticker.replace(".","-")
    if count%10==0:
        print(count)

    accuracy = do_ml(ticker)
    accuracies.append(accuracy)
    print("{} accuracy: {}. Average accuracy:{}".format(ticker,accuracy,mean(accuracies)))

0
Data spread: Counter({'-1': 921, '1': 905, '0': 804})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.5304878048780488
predicted class counts: Counter({-1: 356, 1: 168, 0: 132})


ZTS accuracy: 0.5304878048780488. Average accuracy:0.5304878048780488
Data spread: Counter({'-1': 921, '1': 905, '0': 804})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.5548780487804879
predicted class counts: Counter({-1: 357, 1: 159, 0: 140})


ZTS accuracy: 0.5548780487804879. Average accuracy:0.5426829268292683
Data spread: Counter({'-1': 921, '1': 905, '0': 804})


...

accuracy: 0.510670731707317
predicted class counts: Counter({-1: 386, 1: 160, 0: 110})


ZTS accuracy: 0.510670731707317. Average accuracy:0.5175813008130081
Data spread: Counter({'-1': 921, '1': 905, '0': 804})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.5060975609756098
predicted class counts: Counter({-1: 324, 1: 205, 0: 127})


ZTS accuracy: 0.5060975609756098. Average accuracy:0.5174301989730423
Data spread: Counter({'-1': 921, '1': 905, '0': 804})


c:\users\jk\anaconda3\envs\trade\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy: 0.5015243902439024
predicted class counts: Counter({-1: 348, 1: 186, 0: 122})


ZTS accuracy: 0.5015243902439024. Average accuracy:0.5172236300285081
Data spread: Counter({'-1': 921, '1': 905, '0': 804})
